## **Instalação dos Pacotes**

In [ ]:
# !pip install -U -q langchain langchain-core langchain-openai langchain-community pydantic

## **Configuração das Variáveis de Ambiente**

Configuração da chave de API da OpenAI para autenticação

In [1]:
import os
from pathlib import Path


def load_env() -> None:
    """Load environment variables from the appropriate source.
    
    If running in Google Colab, loads secrets from Colab's userdata.
    Otherwise, loads variables from a local .env file using python-dotenv.
    """
    try:
        import google.colab
        from google.colab import userdata

        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_KEY")

    except ImportError:
        from dotenv import load_dotenv
        load_dotenv()


load_env()

## **Imports dos Pacotes Usados no Notebook**

Importação dos componentes principais do LangChain para construção de aplicações com LLMs

In [2]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

## **Inicialização do Modelo LLM**

Criação da instância do modelo ChatGPT que será utilizado em todos os exemplos

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4.1-nano")

print("Modelo LLM inicializado com sucesso!")

Modelo LLM inicializado com sucesso!


In [4]:
llm.invoke("olá, tudo bem?")

AIMessage(content='Olá! Tudo ótimo, obrigado. Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_336de151a4', 'id': 'chatcmpl-DDiMhPdHpHfFhhMZlUDI0Se9T1apL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9d09-85b1-7fe1-b486-8ec3427a79a9-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
file_url = "https://raw.githubusercontent.com/renansantosmendes/synapseai_data/refs/heads/main/company_context.txt"

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(file_url, encoding="utf-8")
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from pydantic import BaseModel, Field
from typing import List, Dict

class ProductAnalysis(BaseModel):
    """Associação de um produto aos seus problemas."""
    product_name: str = Field(description="Nome do produto")
    problems: List[str] = Field(description="Lista de problemas identificados para este produto")

class ProductsProblems(BaseModel):
    """Estrutura final para a extração de múltiplos produtos."""
    content: List[ProductAnalysis] = Field(description="Lista de produtos e seus problemas")

In [8]:
llm_structured = llm.with_structured_output(ProductsProblems)

In [9]:
products_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes da empresa gere a descrição do
produto e seu nome. O nome deve ser o mais curto possível, sem adicionar a descrição.
O campo descrição deve ser feito de forma separada do nome do produto

A descrição deve conter todos os detalhes técnicos e não técnicos possíveis sobre o produto em questão.

Garanta que nenhuma informação seja criada, use somente o contexto da empresa para extrair as informações
</context>

<company_description>
{company_description}
</company_description>
""")

In [10]:
class ProductEntry(BaseModel):
    """Representa um produto individual da companhia e sua definição oficial."""
    name: str = Field(
        description="O nome oficial do produto (ex: 'Cartão Platinum', 'App Invest', 'CDB Fácil')"
    )
    description: str = Field(
        description="A descrição detalhada e técnica de todas das funcionalidades e regras do produto encontrada no texto"
    )

class CompanyProductCatalog(BaseModel):
    """Catálogo completo de produtos extraído dos documentos da empresa."""
    products: List[ProductEntry] = Field(description="Lista de todos os produtos identificados no texto contendo suas descrições detalhadas")

In [18]:
structured_llm = llm.with_structured_output(CompanyProductCatalog, method="function_calling")
product_chain = products_prompt | structured_llm

## Criação da lista de produtos + descrições

In [19]:
catalog = product_chain.invoke(
    {
        "company_description": docs[0].page_content
    }
)

In [20]:
catalog.products

[ProductEntry(name='FinBrain', description='O FinBrain é um copiloto de IA generativa especializado em análise financeira, contábil e estratégica. Ele foi projetado para apoiar executivos, analistas e equipes financeiras na interpretação de grandes volumes de dados financeiros internos. O produto permite que usuários façam perguntas em linguagem natural sobre balanços, DREs, fluxo de caixa, orçamentos e indicadores financeiros, recebendo respostas claras, explicáveis e rastreáveis. Além disso, o FinBrain oferece simulações de cenários, análises comparativas e geração automática de relatórios executivos. O FinBrain atua sempre como copiloto, mantendo o controle e a decisão final nas mãos dos profissionais.'),
 ProductEntry(name='RiskGen', description='O RiskGen é uma plataforma de IA generativa voltada para gestão de riscos, compliance regulatório e auditoria corporativa. Seu foco está na leitura, interpretação e comparação de documentos complexos, como contratos, políticas internas e n

In [28]:
catalog_dict = {item['name']: item['description'] for item in catalog.model_dump()["products"]}

In [29]:
current_product = "FinBrain"
catalog_dict.get(current_product)

'O FinBrain é um copiloto de IA generativa especializado em análise financeira, contábil e estratégica. Ele foi projetado para apoiar executivos, analistas e equipes financeiras na interpretação de grandes volumes de dados financeiros internos. O produto permite que usuários façam perguntas em linguagem natural sobre balanços, DREs, fluxo de caixa, orçamentos e indicadores financeiros, recebendo respostas claras, explicáveis e rastreáveis. Além disso, o FinBrain oferece simulações de cenários, análises comparativas e geração automática de relatórios executivos. O FinBrain atua sempre como copiloto, mantendo o controle e a decisão final nas mãos dos profissionais.'

## Criação do público de cada produto

In [30]:
class CustomerAnalysis(BaseModel):
    """Análise detalhada de um cliente potencial para um produto."""
    customer_name: str = Field(description="Nome do cliente")
    customer_description: str = Field(description="Descrição detalhada do cliente")

class PublicTargetAnalysis(BaseModel):
    """Análise detalhada do público-alvo de um produto, incluindo jornada do cliente e exemplos reais."""
    target_audience_description: str = Field(description="Descrição detalhada do público-alvo para o produto")
    customer_journey: str = Field(description="Descrição da jornada de uso do produto, desde o primeiro contato até o uso recorrente")
    example_customers: List[CustomerAnalysis] = Field(
        description="Lista de possíveis clientes reais que se encaixariam no público-alvo, com breve descrição de cada um"
    )

In [31]:
customer_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes da empresa gere uma descrição
de um público alvo para o produto, uma possivel jornada de uso do produto, 
desde o primeiro contato do cliente com o produto, até o uso recorrente do mesmo.
Além disso, gere uma lista de possíveis clientes que sejam reais e que se encaixariam 
no público alvo do produto com uma breve descrição de cada um desses clientes.
</context>

<company_description>
{company_description}
</company_description>

Considere o produto "{product_name}" e sua descrição "{product_description}" para conseguir gerar problemas mais
contextualizados
""")

In [25]:
customer_structured_llm = llm.with_structured_output(PublicTargetAnalysis, method="function_calling")
customer_chain = customer_prompt | customer_structured_llm

In [32]:
customers = customer_chain.invoke(
    {
        "company_description": docs[0].page_content,
        "product_name": current_product,
        "product_description": catalog_dict.get(current_product)
    }
)

In [33]:
customers.model_dump().get("example_customers")

[{'customer_name': 'Gerente Financeiro de uma grande rede de varejo',
  'customer_description': 'Responsável por gerenciar o planejamento financeiro, análise de resultados e elaboração de relatórios para uma cadeia de lojas com centenas de unidades, buscando maior agilidade e precisão nas análises financeiras.'},
 {'customer_name': 'Controller de uma multinacional do setor de energia',
  'customer_description': 'Encarregado de controlar e consolidar relatórios financeiros de diversas subsidiárias, interessado em simulações de cenários e controle de riscos financeiros para otimizar processos de tomada de decisão.'},
 {'customer_name': 'Diretor de Planejamento Estratégico de uma instituição financeira',
  'customer_description': 'Foca no alinhamento do planejamento financeiro com as metas corporativas, usando análises preditivas e acompanhamento de indicadores financeiros chave para tomar decisões estratégicas precisas.'},
 {'customer_name': 'Analista de Business Intelligence em uma comp

# usar esse for aqui

In [ ]:
for current_product in catalog_dict.keys():
    print(current_product)

## Criação dos problemas para cada produto

In [34]:
problems_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes de uma empresa gere uma lista de
possíveis problemas que cada produto da empresa pode apresentar do ponto de
vista do usuário final, como por exemplo API instável, sistema demora muito
para responder,
</context>

<company_description>
{company_description}
</company_description>

Use o contexto do produto "{product_name}" e sua descrição "{product_description}" para conseguir criar cenários de problemas reais para 
a abertura de tickets de suporte contextualizados. Garanta que os problemas listados sejam realmente relacionados ao produto e não sejam 
problemas genéricos de mercado.

Use as informações a seguir para conseguir gerar os problemas de cada produto:

## Informações sobre o produto:
- Descrição do público-alvo: {target_audience_description}
- Jornada do cliente: {customer_journey}
- Exemplos de clientes: {example_customers}

Crie problemas especificos para o produto em questão. 

Gere dados que evidencie o problema para cada um dos problemas listados, como por exemplo: porcentagem de clientes afetados, 
impacto financeiro, quantidade de vezes que o problema aconteceu, horários nos quais os problemas ocorrem, etc. 

Use termos técnicos e específicos do domínio do produto para descrever os problemas, evitando generalizações e garantindo que cada problema seja claramente relacionado ao produto em questão.
""")

In [35]:
problems_structured_llm = llm.with_structured_output(ProductsProblems, method="function_calling")
problems_chain = problems_prompt | problems_structured_llm

In [36]:
problems_response = problems_chain.invoke({
    "product_name" : current_product,
    "product_description": catalog_dict.get(current_product),
    "company_description": docs[0].page_content,
    "target_audience_description": customers.model_dump().get("target_audience_description"),
    "customer_journey": customers.model_dump().get("customer_journey"),
    "example_customers": customers.model_dump().get("example_customers"),
})

In [37]:
problems_response.model_dump()["content"]

[{'product_name': 'FinBrain',
  'problems': ['Respostas de perguntas financeiras em linguagem natural sendo incoerentes ou imprecisas, especialmente ao interpretar balanços complexos ou fluxos de caixa detalhados.',
   'Latência na geração de respostas para consultas financeiras complexas, chegando a ultrapassar 10 segundos em 65% dos casos, o que prejudica a tomada de decisão rápida.',
   'Problemas de integração com fontes de dados internas, ocasionando falhas no acesso a ERP e data warehouse, impactando 40% dos usuários em processos de análise de cenários.',
   'Dificuldade no rastreamento de origem das respostas alimentadas por RAG, levando a dúvidas na verificabilidade dos insights para 70% dos analistas financeiros.',
   'Sistema apresenta quedas ou falhas na API REST/GraphQL durante horários de pico de uso (das 9h às 11h e das 14h às 16h), impactando 55% dos usuários na geração de relatórios estratégicos.',
   'Inconsistências na explicabilidade das respostas geradas, com casos 

In [38]:
product_problems_dict = {item['product_name']: item['problems'] for item in problems_response.model_dump()["content"]}

In [39]:
product_problems_dict

{'FinBrain': ['Respostas de perguntas financeiras em linguagem natural sendo incoerentes ou imprecisas, especialmente ao interpretar balanços complexos ou fluxos de caixa detalhados.',
  'Latência na geração de respostas para consultas financeiras complexas, chegando a ultrapassar 10 segundos em 65% dos casos, o que prejudica a tomada de decisão rápida.',
  'Problemas de integração com fontes de dados internas, ocasionando falhas no acesso a ERP e data warehouse, impactando 40% dos usuários em processos de análise de cenários.',
  'Dificuldade no rastreamento de origem das respostas alimentadas por RAG, levando a dúvidas na verificabilidade dos insights para 70% dos analistas financeiros.',
  'Sistema apresenta quedas ou falhas na API REST/GraphQL durante horários de pico de uso (das 9h às 11h e das 14h às 16h), impactando 55% dos usuários na geração de relatórios estratégicos.',
  'Inconsistências na explicabilidade das respostas geradas, com casos em que as referências às fontes orig

In [40]:
import random

def get_random_element(input_list):
    """Retorna um elemento aleatório de uma lista."""
    return random.choice(input_list) if input_list else None

# Exemplo de uso com seus produtos:
meus_produtos = ["App Invest", "Cartão Platinum", "CDB Fácil", "Seguro Vida"]

# Toda vez que chamar, virá um diferente:
produto_sorteado = get_random_element(meus_produtos)
print(f"Produto escolhido: {produto_sorteado}")

Produto escolhido: App Invest


In [41]:
get_random_element(product_problems_dict.get(current_product))

'Problemas de integração com fontes de dados internas, ocasionando falhas no acesso a ERP e data warehouse, impactando 40% dos usuários em processos de análise de cenários.'

In [ ]:
import pandas as pd
import json
from enum import Enum
from typing import List
from datetime import datetime
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


class CriticalityLevel(str, Enum):
    LOW = "Baixa"
    MEDIUM = "Média"
    HIGH = "Alta"
    CRITICAL = "Crítica"

class SupportTicket(BaseModel):
    """Estrutura de um ticket de suporte com dados realistas."""
    customer_name: str = Field(description="Nome completo real (ex: João Carlos de Alencar, Maria Aparecida Souza)")
    customer_email: str = Field(description="E-mail variado (NÃO use @example.com. Use @gmail, @uol, @empresa.com.br, etc)")
    product_name: str = Field(description="Nome do produto do mercado financeiro ou tecnologia")
    issue_description: str = Field(description="Relato do problema em linguagem natural humana")
    opening_date: str = Field(description="Data de abertura (formato ISO YYYY-MM-DD)")
    criticality: CriticalityLevel = Field(description="Nível de urgência")

class TicketBatch(BaseModel):
    """Lote de tickets para exportação."""
    tickets: List[SupportTicket]

structured_llm = llm.with_structured_output(TicketBatch, method="function_calling")

prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um gerador de dados sintéticos de alta fidelidade para o mercado brasileiro.

    DIRETRIZES PARA REALISMO:
    - NOMES: Use nomes brasileiros reais e diversos (sobrenomes como Silva, Oliveira, Cavalcanti, Schmidt, Nogueira).
    - E-MAILS: Use domínios como @hotmail.com, @uol.com.br, @terra.com.br, @gmail.com ou domínios de empresas fictícias (@startup.io, @banco.com.br).
    - PROIBIÇÃO: Está proibido o uso do domínio '@example.com'.
    - DATAS: Gere datas aleatórias no ano de 2025.
    - PERSONAS: Varie entre clientes corporativos sérios e usuários comuns informais."""),
    ("human", "Gere {number_of_tickets} tickets de suporte para o produto '{product_name}' com a seguinte descrição '{product_description}' sobre o problema: '{product_problem}'.")
])


chain = prompt | structured_llm

input_data = {
    "number_of_tickets": 1,
    "product_name": current_product,
    "product_description": catalog_dict.get(current_product),
    "product_problem": get_random_element(product_problems_dict.get(current_product))
}

resultado = chain.invoke(input_data)
tickets_dicts = [t.model_dump() for t in resultado.tickets]

df = pd.DataFrame(tickets_dicts)

print(df[['customer_name', 'customer_email', 'criticality']])


             customer_name             customer_email            criticality
0  Carlos Alberto Oliveira  carlos.oliveira@gmail.com  CriticalityLevel.HIGH


In [43]:
resultado.tickets[3].model_dump()

IndexError: list index out of range